In [1]:
# Import necessary libraries
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Create input variables for the fuzzy control system based on the analysis document
# Input 1: Traffic Density (0-100%)
traffic_density = ctrl.Antecedent(np.arange(0, 101, 1), 'traffic_density')
# Input 2: Time of Day (0-24 hours)
time_of_day = ctrl.Antecedent(np.arange(0, 25, 1), 'time_of_day')
# Input 3: Weather Condition (0-10)
weather_condition = ctrl.Antecedent(np.arange(0, 11, 1), 'weather_condition')
# Input 4: Vacancy Rate (0-100%)
vacancy_rate = ctrl.Antecedent(np.arange(0, 101, 1), 'vacancy_rate')
# Input 5: User Type (1-5)
user_type = ctrl.Antecedent(np.arange(1, 6, 1), 'user_type')

# Create output variables for the fuzzy control system
# Output 1: Recommended Parking Area (1-5)
recommended_area = ctrl.Consequent(np.arange(1, 6, 1), 'recommended_area')
# Output 2: Estimated Waiting Time (0-30 minutes)
waiting_time = ctrl.Consequent(np.arange(0, 31, 1), 'waiting_time')

# Define membership functions for each input variable
# Traffic Density
traffic_density['low'] = fuzz.trapmf(traffic_density.universe, [0, 0, 20, 40])
traffic_density['medium'] = fuzz.trimf(traffic_density.universe, [30, 50, 70])
traffic_density['high'] = fuzz.trapmf(traffic_density.universe, [60, 80, 100, 100])

# Time of Day
time_of_day['early_morning'] = fuzz.trapmf(time_of_day.universe, [0, 0, 6, 8])
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [7, 9, 11])
time_of_day['noon'] = fuzz.trimf(time_of_day.universe, [10, 12, 14])
time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [13, 15, 18])
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [17, 19, 22])
time_of_day['night'] = fuzz.trapmf(time_of_day.universe, [21, 23, 24, 24])

# Weather Condition
weather_condition['clear'] = fuzz.trapmf(weather_condition.universe, [0, 0, 2, 3])
weather_condition['light_rain'] = fuzz.trimf(weather_condition.universe, [2, 4, 6])
weather_condition['heavy_rain'] = fuzz.trimf(weather_condition.universe, [5, 7, 9])
weather_condition['snow'] = fuzz.trapmf(weather_condition.universe, [8, 9, 10, 10])

# Vacancy Rate
vacancy_rate['very_low'] = fuzz.trapmf(vacancy_rate.universe, [0, 0, 10, 20])
vacancy_rate['low'] = fuzz.trimf(vacancy_rate.universe, [15, 25, 35])
vacancy_rate['medium'] = fuzz.trimf(vacancy_rate.universe, [30, 50, 70])
vacancy_rate['high'] = fuzz.trimf(vacancy_rate.universe, [60, 75, 90])
vacancy_rate['very_high'] = fuzz.trapmf(vacancy_rate.universe, [85, 95, 100, 100])

# User Type
user_type['regular'] = fuzz.trimf(user_type.universe, [1, 1, 2])
user_type['member'] = fuzz.trimf(user_type.universe, [1, 2, 3])
user_type['vip'] = fuzz.trimf(user_type.universe, [2, 3, 4])
user_type['disabled'] = fuzz.trimf(user_type.universe, [3, 4, 5])
user_type['staff'] = fuzz.trimf(user_type.universe, [4, 5, 5])

# Define membership functions for each output variable
# Recommended Parking Area
recommended_area['area_a'] = fuzz.trimf(recommended_area.universe, [1, 1, 2])  # Closest to entrance
recommended_area['area_b'] = fuzz.trimf(recommended_area.universe, [1, 2, 3])
recommended_area['area_c'] = fuzz.trimf(recommended_area.universe, [2, 3, 4])
recommended_area['area_d'] = fuzz.trimf(recommended_area.universe, [3, 4, 5])
recommended_area['area_e'] = fuzz.trimf(recommended_area.universe, [4, 5, 5])  # Farthest from entrance

# Estimated Waiting Time
waiting_time['very_short'] = fuzz.trapmf(waiting_time.universe, [0, 0, 2, 5])
waiting_time['short'] = fuzz.trimf(waiting_time.universe, [3, 7, 11])
waiting_time['medium'] = fuzz.trimf(waiting_time.universe, [9, 13, 17])
waiting_time['long'] = fuzz.trimf(waiting_time.universe, [15, 20, 25])
waiting_time['very_long'] = fuzz.trapmf(waiting_time.universe, [23, 28, 30, 30])

# Define fuzzy rules for the control system

# Rules for Recommended Parking Area
rule1 = ctrl.Rule(vacancy_rate['very_high'], recommended_area['area_a'])
rule2 = ctrl.Rule(vacancy_rate['high'] & user_type['regular'], recommended_area['area_b'])
rule3 = ctrl.Rule(vacancy_rate['high'] & ~user_type['regular'], recommended_area['area_a'])
rule4 = ctrl.Rule(vacancy_rate['medium'] & user_type['regular'], recommended_area['area_c'])
rule5 = ctrl.Rule(vacancy_rate['medium'] & user_type['member'], recommended_area['area_b'])
rule6 = ctrl.Rule(vacancy_rate['medium'] & (user_type['vip'] | user_type['disabled'] | user_type['staff']), 
                recommended_area['area_a'])
rule7 = ctrl.Rule(vacancy_rate['low'] & user_type['regular'], recommended_area['area_d'])
rule8 = ctrl.Rule(vacancy_rate['low'] & user_type['member'], recommended_area['area_c'])
rule9 = ctrl.Rule(vacancy_rate['low'] & user_type['vip'], recommended_area['area_b'])
rule10 = ctrl.Rule(vacancy_rate['low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_a'])
rule11 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['regular'] | user_type['member']), 
                 recommended_area['area_e'])
rule12 = ctrl.Rule(vacancy_rate['very_low'] & user_type['vip'], recommended_area['area_d'])
rule13 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_c'])
rule14 = ctrl.Rule(weather_condition['snow'] & user_type['disabled'], recommended_area['area_a'])
rule15 = ctrl.Rule(weather_condition['heavy_rain'] & user_type['disabled'], recommended_area['area_a'])

# Rules for Estimated Waiting Time
rule16 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_high'], waiting_time['very_short'])
rule17 = ctrl.Rule(traffic_density['low'] & vacancy_rate['high'], waiting_time['very_short'])
rule18 = ctrl.Rule(traffic_density['low'] & vacancy_rate['medium'], waiting_time['short'])
rule19 = ctrl.Rule(traffic_density['low'] & vacancy_rate['low'], waiting_time['medium'])
rule20 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_low'], waiting_time['long'])

rule21 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_high'], waiting_time['short'])
rule22 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['high'], waiting_time['short'])
rule23 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['medium'], waiting_time['medium'])
rule24 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['low'], waiting_time['long'])
rule25 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_low'], waiting_time['very_long'])

rule26 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_high'], waiting_time['medium'])
rule27 = ctrl.Rule(traffic_density['high'] & vacancy_rate['high'], waiting_time['medium'])
rule28 = ctrl.Rule(traffic_density['high'] & vacancy_rate['medium'], waiting_time['long'])
rule29 = ctrl.Rule(traffic_density['high'] & vacancy_rate['low'], waiting_time['very_long'])
rule30 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_low'], waiting_time['very_long'])

# Time-based rules
rule31 = ctrl.Rule((time_of_day['morning'] | time_of_day['afternoon']) & traffic_density['high'], 
                 waiting_time['very_long'])
rule32 = ctrl.Rule(weather_condition['heavy_rain'] | weather_condition['snow'], waiting_time['long'])
rule33 = ctrl.Rule(time_of_day['night'] & ~vacancy_rate['very_low'], waiting_time['very_short'])

rule34 = ctrl.Rule((time_of_day['morning']) & (vacancy_rate['high'] | vacancy_rate['very_high']), 
                 recommended_area['area_a'])
rule35 = ctrl.Rule(time_of_day['evening'] & traffic_density['high'], waiting_time['very_long'])
rule36 = ctrl.Rule(time_of_day['night'], recommended_area['area_a'])

# Create control system
parking_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20,
    rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28, rule29, rule30,
    rule31, rule32, rule33, rule34, rule35, rule36
])

# Create simulator
parking_simulator = ctrl.ControlSystemSimulation(parking_ctrl)

# Function to use the parking guidance system
def get_parking_recommendation(traffic_density_val, time_of_day_val, weather_condition_val, 
                             vacancy_rate_val, user_type_val):
    """
    Get parking recommendation based on input variables
    
    Args:
        traffic_density_val: Traffic density value (0-100%)
        time_of_day_val: Time of day (0-24 hours)
        weather_condition_val: Weather condition (0-10)
        vacancy_rate_val: Vacancy rate (0-100%)
        user_type_val: User type (1-5)
    
    Returns:
        dict: Dictionary containing recommended parking area and estimated waiting time
    """
    # Input values to the fuzzy control system
    parking_simulator.input['traffic_density'] = traffic_density_val
    parking_simulator.input['time_of_day'] = time_of_day_val
    parking_simulator.input['weather_condition'] = weather_condition_val
    parking_simulator.input['vacancy_rate'] = vacancy_rate_val
    parking_simulator.input['user_type'] = user_type_val
    
    # Compute result
    try:
        parking_simulator.compute()
        
        # Get output values
        recommended_area_val = parking_simulator.output['recommended_area']
        waiting_time_val = parking_simulator.output['waiting_time']
        
        # Convert area to human-readable format
        if recommended_area_val <= 1.5:
            area_text = "Area A (closest to entrance)"
        elif recommended_area_val <= 2.5:
            area_text = "Area B"
        elif recommended_area_val <= 3.5:
            area_text = "Area C"
        elif recommended_area_val <= 4.5:
            area_text = "Area D"
        else:
            area_text = "Area E (farthest from entrance)"
        
        return {
            'recommended_area_value': recommended_area_val,
            'recommended_area_text': area_text,
            'waiting_time_value': waiting_time_val,
            'waiting_time_text': f"{int(waiting_time_val)} minutes"
        }
    except:
        return {
            'error': 'Unable to compute recommendation. Please check input values.'
        }

# Test the system with sample inputs
if __name__ == "__main__":
    # Example: Rush hour, high traffic, medium vacancy, regular user
    result = get_parking_recommendation(
        traffic_density_val=80,    # High traffic
        time_of_day_val=9,         # Morning rush hour
        weather_condition_val=1,    # Clear weather
        vacancy_rate_val=50,       # Medium vacancy
        user_type_val=1            # Regular user
    )
    print("Test Result:")
    print(f"Recommended Parking: {result['recommended_area_text']} (value: {result['recommended_area_value']})")
    print(f"Estimated Waiting Time: {result['waiting_time_text']} (value: {result['waiting_time_value']})")

ModuleNotFoundError: No module named 'skfuzzy'

In [2]:
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()
import asyncio

# Install required packages
async def install_packages():
    await terminal.run('pip install scikit-fuzzy')

# Execute the installation
asyncio.run(install_packages())

# Import necessary libraries
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Create input variables for the fuzzy control system based on the analysis document
# Input 1: Traffic Density (0-100%)
traffic_density = ctrl.Antecedent(np.arange(0, 101, 1), 'traffic_density')
# Input 2: Time of Day (0-24 hours)
time_of_day = ctrl.Antecedent(np.arange(0, 25, 1), 'time_of_day')
# Input 3: Weather Condition (0-10)
weather_condition = ctrl.Antecedent(np.arange(0, 11, 1), 'weather_condition')
# Input 4: Vacancy Rate (0-100%)
vacancy_rate = ctrl.Antecedent(np.arange(0, 101, 1), 'vacancy_rate')
# Input 5: User Type (1-5)
user_type = ctrl.Antecedent(np.arange(1, 6, 1), 'user_type')

# Create output variables for the fuzzy control system
# Output 1: Recommended Parking Area (1-5)
recommended_area = ctrl.Consequent(np.arange(1, 6, 1), 'recommended_area')
# Output 2: Estimated Waiting Time (0-30 minutes)
waiting_time = ctrl.Consequent(np.arange(0, 31, 1), 'waiting_time')

# Define membership functions for each input variable
# Traffic Density
traffic_density['low'] = fuzz.trapmf(traffic_density.universe, [0, 0, 20, 40])
traffic_density['medium'] = fuzz.trimf(traffic_density.universe, [30, 50, 70])
traffic_density['high'] = fuzz.trapmf(traffic_density.universe, [60, 80, 100, 100])

# Time of Day
time_of_day['early_morning'] = fuzz.trapmf(time_of_day.universe, [0, 0, 6, 8])
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [7, 9, 11])
time_of_day['noon'] = fuzz.trimf(time_of_day.universe, [10, 12, 14])
time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [13, 15, 18])
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [17, 19, 22])
time_of_day['night'] = fuzz.trapmf(time_of_day.universe, [21, 23, 24, 24])

# Weather Condition
weather_condition['clear'] = fuzz.trapmf(weather_condition.universe, [0, 0, 2, 3])
weather_condition['light_rain'] = fuzz.trimf(weather_condition.universe, [2, 4, 6])
weather_condition['heavy_rain'] = fuzz.trimf(weather_condition.universe, [5, 7, 9])
weather_condition['snow'] = fuzz.trapmf(weather_condition.universe, [8, 9, 10, 10])

# Vacancy Rate
vacancy_rate['very_low'] = fuzz.trapmf(vacancy_rate.universe, [0, 0, 10, 20])
vacancy_rate['low'] = fuzz.trimf(vacancy_rate.universe, [15, 25, 35])
vacancy_rate['medium'] = fuzz.trimf(vacancy_rate.universe, [30, 50, 70])
vacancy_rate['high'] = fuzz.trimf(vacancy_rate.universe, [60, 75, 90])
vacancy_rate['very_high'] = fuzz.trapmf(vacancy_rate.universe, [85, 95, 100, 100])

# User Type
user_type['regular'] = fuzz.trimf(user_type.universe, [1, 1, 2])
user_type['member'] = fuzz.trimf(user_type.universe, [1, 2, 3])
user_type['vip'] = fuzz.trimf(user_type.universe, [2, 3, 4])
user_type['disabled'] = fuzz.trimf(user_type.universe, [3, 4, 5])
user_type['staff'] = fuzz.trimf(user_type.universe, [4, 5, 5])

# Define membership functions for each output variable
# Recommended Parking Area
recommended_area['area_a'] = fuzz.trimf(recommended_area.universe, [1, 1, 2])  # Closest to entrance
recommended_area['area_b'] = fuzz.trimf(recommended_area.universe, [1, 2, 3])
recommended_area['area_c'] = fuzz.trimf(recommended_area.universe, [2, 3, 4])
recommended_area['area_d'] = fuzz.trimf(recommended_area.universe, [3, 4, 5])
recommended_area['area_e'] = fuzz.trimf(recommended_area.universe, [4, 5, 5])  # Farthest from entrance

# Estimated Waiting Time
waiting_time['very_short'] = fuzz.trapmf(waiting_time.universe, [0, 0, 2, 5])
waiting_time['short'] = fuzz.trimf(waiting_time.universe, [3, 7, 11])
waiting_time['medium'] = fuzz.trimf(waiting_time.universe, [9, 13, 17])
waiting_time['long'] = fuzz.trimf(waiting_time.universe, [15, 20, 25])
waiting_time['very_long'] = fuzz.trapmf(waiting_time.universe, [23, 28, 30, 30])

# Define fuzzy rules for the control system

# Rules for Recommended Parking Area
rule1 = ctrl.Rule(vacancy_rate['very_high'], recommended_area['area_a'])
rule2 = ctrl.Rule(vacancy_rate['high'] & user_type['regular'], recommended_area['area_b'])
rule3 = ctrl.Rule(vacancy_rate['high'] & ~user_type['regular'], recommended_area['area_a'])
rule4 = ctrl.Rule(vacancy_rate['medium'] & user_type['regular'], recommended_area['area_c'])
rule5 = ctrl.Rule(vacancy_rate['medium'] & user_type['member'], recommended_area['area_b'])
rule6 = ctrl.Rule(vacancy_rate['medium'] & (user_type['vip'] | user_type['disabled'] | user_type['staff']), 
                recommended_area['area_a'])
rule7 = ctrl.Rule(vacancy_rate['low'] & user_type['regular'], recommended_area['area_d'])
rule8 = ctrl.Rule(vacancy_rate['low'] & user_type['member'], recommended_area['area_c'])
rule9 = ctrl.Rule(vacancy_rate['low'] & user_type['vip'], recommended_area['area_b'])
rule10 = ctrl.Rule(vacancy_rate['low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_a'])
rule11 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['regular'] | user_type['member']), 
                 recommended_area['area_e'])
rule12 = ctrl.Rule(vacancy_rate['very_low'] & user_type['vip'], recommended_area['area_d'])
rule13 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_c'])
rule14 = ctrl.Rule(weather_condition['snow'] & user_type['disabled'], recommended_area['area_a'])
rule15 = ctrl.Rule(weather_condition['heavy_rain'] & user_type['disabled'], recommended_area['area_a'])

# Rules for Estimated Waiting Time
rule16 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_high'], waiting_time['very_short'])
rule17 = ctrl.Rule(traffic_density['low'] & vacancy_rate['high'], waiting_time['very_short'])
rule18 = ctrl.Rule(traffic_density['low'] & vacancy_rate['medium'], waiting_time['short'])
rule19 = ctrl.Rule(traffic_density['low'] & vacancy_rate['low'], waiting_time['medium'])
rule20 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_low'], waiting_time['long'])

rule21 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_high'], waiting_time['short'])
rule22 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['high'], waiting_time['short'])
rule23 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['medium'], waiting_time['medium'])
rule24 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['low'], waiting_time['long'])
rule25 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_low'], waiting_time['very_long'])

rule26 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_high'], waiting_time['medium'])
rule27 = ctrl.Rule(traffic_density['high'] & vacancy_rate['high'], waiting_time['medium'])
rule28 = ctrl.Rule(traffic_density['high'] & vacancy_rate['medium'], waiting_time['long'])
rule29 = ctrl.Rule(traffic_density['high'] & vacancy_rate['low'], waiting_time['very_long'])
rule30 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_low'], waiting_time['very_long'])

# Time-based rules
rule31 = ctrl.Rule((time_of_day['morning'] | time_of_day['afternoon']) & traffic_density['high'], 
                 waiting_time['very_long'])
rule32 = ctrl.Rule(weather_condition['heavy_rain'] | weather_condition['snow'], waiting_time['long'])
rule33 = ctrl.Rule(time_of_day['night'] & ~vacancy_rate['very_low'], waiting_time['very_short'])

rule34 = ctrl.Rule((time_of_day['morning']) & (vacancy_rate['high'] | vacancy_rate['very_high']), 
                 recommended_area['area_a'])
rule35 = ctrl.Rule(time_of_day['evening'] & traffic_density['high'], waiting_time['very_long'])
rule36 = ctrl.Rule(time_of_day['night'], recommended_area['area_a'])

# Create control system
parking_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20,
    rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28, rule29, rule30,
    rule31, rule32, rule33, rule34, rule35, rule36
])

# Create simulator
parking_simulator = ctrl.ControlSystemSimulation(parking_ctrl)

# Function to use the parking guidance system
def get_parking_recommendation(traffic_density_val, time_of_day_val, weather_condition_val, 
                             vacancy_rate_val, user_type_val):
    """
    Get parking recommendation based on input variables
    
    Args:
        traffic_density_val: Traffic density value (0-100%)
        time_of_day_val: Time of day (0-24 hours)
        weather_condition_val: Weather condition (0-10)
        vacancy_rate_val: Vacancy rate (0-100%)
        user_type_val: User type (1-5)
    
    Returns:
        dict: Dictionary containing recommended parking area and estimated waiting time
    """
    # Input values to the fuzzy control system
    parking_simulator.input['traffic_density'] = traffic_density_val
    parking_simulator.input['time_of_day'] = time_of_day_val
    parking_simulator.input['weather_condition'] = weather_condition_val
    parking_simulator.input['vacancy_rate'] = vacancy_rate_val
    parking_simulator.input['user_type'] = user_type_val
    
    # Compute result
    try:
        parking_simulator.compute()
        
        # Get output values
        recommended_area_val = parking_simulator.output['recommended_area']
        waiting_time_val = parking_simulator.output['waiting_time']
        
        # Convert area to human-readable format
        if recommended_area_val <= 1.5:
            area_text = "Area A (closest to entrance)"
        elif recommended_area_val <= 2.5:
            area_text = "Area B"
        elif recommended_area_val <= 3.5:
            area_text = "Area C"
        elif recommended_area_val <= 4.5:
            area_text = "Area D"
        else:
            area_text = "Area E (farthest from entrance)"
        
        return {
            'recommended_area_value': recommended_area_val,
            'recommended_area_text': area_text,
            'waiting_time_value': waiting_time_val,
            'waiting_time_text': f"{int(waiting_time_val)} minutes"
        }
    except:
        return {
            'error': 'Unable to compute recommendation. Please check input values.'
        }

# Test the system with sample inputs
if __name__ == "__main__":
    # Example: Rush hour, high traffic, medium vacancy, regular user
    result = get_parking_recommendation(
        traffic_density_val=80,    # High traffic
        time_of_day_val=9,         # Morning rush hour
        weather_condition_val=1,    # Clear weather
        vacancy_rate_val=50,       # Medium vacancy
        user_type_val=1            # Regular user
    )
    print("Test Result:")
    print(f"Recommended Parking: {result['recommended_area_text']} (value: {result['recommended_area_value']})")
    print(f"Estimated Waiting Time: {result['waiting_time_text']} (value: {result['waiting_time_value']})")

RuntimeError: asyncio.run() cannot be called from a running event loop

In [3]:
# Import necessary libraries for package installation
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Install the required package
await terminal.run('pip install scikit-fuzzy')

# Now import the required libraries
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Create input variables for the fuzzy control system based on the analysis document
# Input 1: Traffic Density (0-100%)
traffic_density = ctrl.Antecedent(np.arange(0, 101, 1), 'traffic_density')
# Input 2: Time of Day (0-24 hours)
time_of_day = ctrl.Antecedent(np.arange(0, 25, 1), 'time_of_day')
# Input 3: Weather Condition (0-10)
weather_condition = ctrl.Antecedent(np.arange(0, 11, 1), 'weather_condition')
# Input 4: Vacancy Rate (0-100%)
vacancy_rate = ctrl.Antecedent(np.arange(0, 101, 1), 'vacancy_rate')
# Input 5: User Type (1-5)
user_type = ctrl.Antecedent(np.arange(1, 6, 1), 'user_type')

# Create output variables for the fuzzy control system
# Output 1: Recommended Parking Area (1-5)
recommended_area = ctrl.Consequent(np.arange(1, 6, 1), 'recommended_area')
# Output 2: Estimated Waiting Time (0-30 minutes)
waiting_time = ctrl.Consequent(np.arange(0, 31, 1), 'waiting_time')

# Define membership functions for each input variable
# Traffic Density
traffic_density['low'] = fuzz.trapmf(traffic_density.universe, [0, 0, 20, 40])
traffic_density['medium'] = fuzz.trimf(traffic_density.universe, [30, 50, 70])
traffic_density['high'] = fuzz.trapmf(traffic_density.universe, [60, 80, 100, 100])

# Time of Day
time_of_day['early_morning'] = fuzz.trapmf(time_of_day.universe, [0, 0, 6, 8])
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [7, 9, 11])
time_of_day['noon'] = fuzz.trimf(time_of_day.universe, [10, 12, 14])
time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [13, 15, 18])
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [17, 19, 22])
time_of_day['night'] = fuzz.trapmf(time_of_day.universe, [21, 23, 24, 24])

# Weather Condition
weather_condition['clear'] = fuzz.trapmf(weather_condition.universe, [0, 0, 2, 3])
weather_condition['light_rain'] = fuzz.trimf(weather_condition.universe, [2, 4, 6])
weather_condition['heavy_rain'] = fuzz.trimf(weather_condition.universe, [5, 7, 9])
weather_condition['snow'] = fuzz.trapmf(weather_condition.universe, [8, 9, 10, 10])

# Vacancy Rate
vacancy_rate['very_low'] = fuzz.trapmf(vacancy_rate.universe, [0, 0, 10, 20])
vacancy_rate['low'] = fuzz.trimf(vacancy_rate.universe, [15, 25, 35])
vacancy_rate['medium'] = fuzz.trimf(vacancy_rate.universe, [30, 50, 70])
vacancy_rate['high'] = fuzz.trimf(vacancy_rate.universe, [60, 75, 90])
vacancy_rate['very_high'] = fuzz.trapmf(vacancy_rate.universe, [85, 95, 100, 100])

# User Type
user_type['regular'] = fuzz.trimf(user_type.universe, [1, 1, 2])
user_type['member'] = fuzz.trimf(user_type.universe, [1, 2, 3])
user_type['vip'] = fuzz.trimf(user_type.universe, [2, 3, 4])
user_type['disabled'] = fuzz.trimf(user_type.universe, [3, 4, 5])
user_type['staff'] = fuzz.trimf(user_type.universe, [4, 5, 5])

# Define membership functions for each output variable
# Recommended Parking Area
recommended_area['area_a'] = fuzz.trimf(recommended_area.universe, [1, 1, 2])  # Closest to entrance
recommended_area['area_b'] = fuzz.trimf(recommended_area.universe, [1, 2, 3])
recommended_area['area_c'] = fuzz.trimf(recommended_area.universe, [2, 3, 4])
recommended_area['area_d'] = fuzz.trimf(recommended_area.universe, [3, 4, 5])
recommended_area['area_e'] = fuzz.trimf(recommended_area.universe, [4, 5, 5])  # Farthest from entrance

# Estimated Waiting Time
waiting_time['very_short'] = fuzz.trapmf(waiting_time.universe, [0, 0, 2, 5])
waiting_time['short'] = fuzz.trimf(waiting_time.universe, [3, 7, 11])
waiting_time['medium'] = fuzz.trimf(waiting_time.universe, [9, 13, 17])
waiting_time['long'] = fuzz.trimf(waiting_time.universe, [15, 20, 25])
waiting_time['very_long'] = fuzz.trapmf(waiting_time.universe, [23, 28, 30, 30])

# Define fuzzy rules for the control system

# Rules for Recommended Parking Area
rule1 = ctrl.Rule(vacancy_rate['very_high'], recommended_area['area_a'])
rule2 = ctrl.Rule(vacancy_rate['high'] & user_type['regular'], recommended_area['area_b'])
rule3 = ctrl.Rule(vacancy_rate['high'] & ~user_type['regular'], recommended_area['area_a'])
rule4 = ctrl.Rule(vacancy_rate['medium'] & user_type['regular'], recommended_area['area_c'])
rule5 = ctrl.Rule(vacancy_rate['medium'] & user_type['member'], recommended_area['area_b'])
rule6 = ctrl.Rule(vacancy_rate['medium'] & (user_type['vip'] | user_type['disabled'] | user_type['staff']), 
                recommended_area['area_a'])
rule7 = ctrl.Rule(vacancy_rate['low'] & user_type['regular'], recommended_area['area_d'])
rule8 = ctrl.Rule(vacancy_rate['low'] & user_type['member'], recommended_area['area_c'])
rule9 = ctrl.Rule(vacancy_rate['low'] & user_type['vip'], recommended_area['area_b'])
rule10 = ctrl.Rule(vacancy_rate['low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_a'])
rule11 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['regular'] | user_type['member']), 
                 recommended_area['area_e'])
rule12 = ctrl.Rule(vacancy_rate['very_low'] & user_type['vip'], recommended_area['area_d'])
rule13 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_c'])
rule14 = ctrl.Rule(weather_condition['snow'] & user_type['disabled'], recommended_area['area_a'])
rule15 = ctrl.Rule(weather_condition['heavy_rain'] & user_type['disabled'], recommended_area['area_a'])

# Rules for Estimated Waiting Time
rule16 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_high'], waiting_time['very_short'])
rule17 = ctrl.Rule(traffic_density['low'] & vacancy_rate['high'], waiting_time['very_short'])
rule18 = ctrl.Rule(traffic_density['low'] & vacancy_rate['medium'], waiting_time['short'])
rule19 = ctrl.Rule(traffic_density['low'] & vacancy_rate['low'], waiting_time['medium'])
rule20 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_low'], waiting_time['long'])

rule21 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_high'], waiting_time['short'])
rule22 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['high'], waiting_time['short'])
rule23 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['medium'], waiting_time['medium'])
rule24 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['low'], waiting_time['long'])
rule25 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_low'], waiting_time['very_long'])

rule26 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_high'], waiting_time['medium'])
rule27 = ctrl.Rule(traffic_density['high'] & vacancy_rate['high'], waiting_time['medium'])
rule28 = ctrl.Rule(traffic_density['high'] & vacancy_rate['medium'], waiting_time['long'])
rule29 = ctrl.Rule(traffic_density['high'] & vacancy_rate['low'], waiting_time['very_long'])
rule30 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_low'], waiting_time['very_long'])

# Time-based rules
rule31 = ctrl.Rule((time_of_day['morning'] | time_of_day['afternoon']) & traffic_density['high'], 
                 waiting_time['very_long'])
rule32 = ctrl.Rule(weather_condition['heavy_rain'] | weather_condition['snow'], waiting_time['long'])
rule33 = ctrl.Rule(time_of_day['night'] & ~vacancy_rate['very_low'], waiting_time['very_short'])

rule34 = ctrl.Rule((time_of_day['morning']) & (vacancy_rate['high'] | vacancy_rate['very_high']), 
                 recommended_area['area_a'])
rule35 = ctrl.Rule(time_of_day['evening'] & traffic_density['high'], waiting_time['very_long'])
rule36 = ctrl.Rule(time_of_day['night'], recommended_area['area_a'])

# Create control system
parking_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20,
    rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28, rule29, rule30,
    rule31, rule32, rule33, rule34, rule35, rule36
])

# Create simulator
parking_simulator = ctrl.ControlSystemSimulation(parking_ctrl)

# Function to use the parking guidance system
def get_parking_recommendation(traffic_density_val, time_of_day_val, weather_condition_val, 
                             vacancy_rate_val, user_type_val):
    """
    Get parking recommendation based on input variables
    
    Args:
        traffic_density_val: Traffic density value (0-100%)
        time_of_day_val: Time of day (0-24 hours)
        weather_condition_val: Weather condition (0-10)
        vacancy_rate_val: Vacancy rate (0-100%)
        user_type_val: User type (1-5)
    
    Returns:
        dict: Dictionary containing recommended parking area and estimated waiting time
    """
    # Input values to the fuzzy control system
    parking_simulator.input['traffic_density'] = traffic_density_val
    parking_simulator.input['time_of_day'] = time_of_day_val
    parking_simulator.input['weather_condition'] = weather_condition_val
    parking_simulator.input['vacancy_rate'] = vacancy_rate_val
    parking_simulator.input['user_type'] = user_type_val
    
    # Compute result
    try:
        parking_simulator.compute()
        
        # Get output values
        recommended_area_val = parking_simulator.output['recommended_area']
        waiting_time_val = parking_simulator.output['waiting_time']
        
        # Convert area to human-readable format
        if recommended_area_val <= 1.5:
            area_text = "Area A (closest to entrance)"
        elif recommended_area_val <= 2.5:
            area_text = "Area B"
        elif recommended_area_val <= 3.5:
            area_text = "Area C"
        elif recommended_area_val <= 4.5:
            area_text = "Area D"
        else:
            area_text = "Area E (farthest from entrance)"
        
        return {
            'recommended_area_value': recommended_area_val,
            'recommended_area_text': area_text,
            'waiting_time_value': waiting_time_val,
            'waiting_time_text': f"{int(waiting_time_val)} minutes"
        }
    except:
        return {
            'error': 'Unable to compute recommendation. Please check input values.'
        }

# Test the system with sample inputs
if __name__ == "__main__":
    # Example: Rush hour, high traffic, medium vacancy, regular user
    result = get_parking_recommendation(
        traffic_density_val=80,    # High traffic
        time_of_day_val=9,         # Morning rush hour
        weather_condition_val=1,    # Clear weather
        vacancy_rate_val=50,       # Medium vacancy
        user_type_val=1            # Regular user
    )
    print("Test Result:")
    print(f"Recommended Parking: {result['recommended_area_text']} (value: {result['recommended_area_value']})")
    print(f"Estimated Waiting Time: {result['waiting_time_text']} (value: {result['waiting_time_value']})")

ModuleNotFoundError: No module named 'skfuzzy'

In [4]:
# Import necessary libraries
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Create input variables for the fuzzy control system based on the analysis document
# Input 1: Traffic Density (0-100%)
traffic_density = ctrl.Antecedent(np.arange(0, 101, 1), 'traffic_density')
# Input 2: Time of Day (0-24 hours)
time_of_day = ctrl.Antecedent(np.arange(0, 25, 1), 'time_of_day')
# Input 3: Weather Condition (0-10)
weather_condition = ctrl.Antecedent(np.arange(0, 11, 1), 'weather_condition')
# Input 4: Vacancy Rate (0-100%)
vacancy_rate = ctrl.Antecedent(np.arange(0, 101, 1), 'vacancy_rate')
# Input 5: User Type (1-5)
user_type = ctrl.Antecedent(np.arange(1, 6, 1), 'user_type')

# Create output variables for the fuzzy control system
# Output 1: Recommended Parking Area (1-5)
recommended_area = ctrl.Consequent(np.arange(1, 6, 1), 'recommended_area')
# Output 2: Estimated Waiting Time (0-30 minutes)
waiting_time = ctrl.Consequent(np.arange(0, 31, 1), 'waiting_time')

# Define membership functions for each input variable
# Traffic Density
traffic_density['low'] = fuzz.trapmf(traffic_density.universe, [0, 0, 20, 40])
traffic_density['medium'] = fuzz.trimf(traffic_density.universe, [30, 50, 70])
traffic_density['high'] = fuzz.trapmf(traffic_density.universe, [60, 80, 100, 100])

# Time of Day
time_of_day['early_morning'] = fuzz.trapmf(time_of_day.universe, [0, 0, 6, 8])
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [7, 9, 11])
time_of_day['noon'] = fuzz.trimf(time_of_day.universe, [10, 12, 14])
time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [13, 15, 18])
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [17, 19, 22])
time_of_day['night'] = fuzz.trapmf(time_of_day.universe, [21, 23, 24, 24])

# Weather Condition
weather_condition['clear'] = fuzz.trapmf(weather_condition.universe, [0, 0, 2, 3])
weather_condition['light_rain'] = fuzz.trimf(weather_condition.universe, [2, 4, 6])
weather_condition['heavy_rain'] = fuzz.trimf(weather_condition.universe, [5, 7, 9])
weather_condition['snow'] = fuzz.trapmf(weather_condition.universe, [8, 9, 10, 10])

# Vacancy Rate
vacancy_rate['very_low'] = fuzz.trapmf(vacancy_rate.universe, [0, 0, 10, 20])
vacancy_rate['low'] = fuzz.trimf(vacancy_rate.universe, [15, 25, 35])
vacancy_rate['medium'] = fuzz.trimf(vacancy_rate.universe, [30, 50, 70])
vacancy_rate['high'] = fuzz.trimf(vacancy_rate.universe, [60, 75, 90])
vacancy_rate['very_high'] = fuzz.trapmf(vacancy_rate.universe, [85, 95, 100, 100])

# User Type
user_type['regular'] = fuzz.trimf(user_type.universe, [1, 1, 2])
user_type['member'] = fuzz.trimf(user_type.universe, [1, 2, 3])
user_type['vip'] = fuzz.trimf(user_type.universe, [2, 3, 4])
user_type['disabled'] = fuzz.trimf(user_type.universe, [3, 4, 5])
user_type['staff'] = fuzz.trimf(user_type.universe, [4, 5, 5])

# Define membership functions for each output variable
# Recommended Parking Area
recommended_area['area_a'] = fuzz.trimf(recommended_area.universe, [1, 1, 2])  # Closest to entrance
recommended_area['area_b'] = fuzz.trimf(recommended_area.universe, [1, 2, 3])
recommended_area['area_c'] = fuzz.trimf(recommended_area.universe, [2, 3, 4])
recommended_area['area_d'] = fuzz.trimf(recommended_area.universe, [3, 4, 5])
recommended_area['area_e'] = fuzz.trimf(recommended_area.universe, [4, 5, 5])  # Farthest from entrance

# Estimated Waiting Time
waiting_time['very_short'] = fuzz.trapmf(waiting_time.universe, [0, 0, 2, 5])
waiting_time['short'] = fuzz.trimf(waiting_time.universe, [3, 7, 11])
waiting_time['medium'] = fuzz.trimf(waiting_time.universe, [9, 13, 17])
waiting_time['long'] = fuzz.trimf(waiting_time.universe, [15, 20, 25])
waiting_time['very_long'] = fuzz.trapmf(waiting_time.universe, [23, 28, 30, 30])

# Define fuzzy rules for the control system

# Rules for Recommended Parking Area
rule1 = ctrl.Rule(vacancy_rate['very_high'], recommended_area['area_a'])
rule2 = ctrl.Rule(vacancy_rate['high'] & user_type['regular'], recommended_area['area_b'])
rule3 = ctrl.Rule(vacancy_rate['high'] & ~user_type['regular'], recommended_area['area_a'])
rule4 = ctrl.Rule(vacancy_rate['medium'] & user_type['regular'], recommended_area['area_c'])
rule5 = ctrl.Rule(vacancy_rate['medium'] & user_type['member'], recommended_area['area_b'])
rule6 = ctrl.Rule(vacancy_rate['medium'] & (user_type['vip'] | user_type['disabled'] | user_type['staff']), 
                recommended_area['area_a'])
rule7 = ctrl.Rule(vacancy_rate['low'] & user_type['regular'], recommended_area['area_d'])
rule8 = ctrl.Rule(vacancy_rate['low'] & user_type['member'], recommended_area['area_c'])
rule9 = ctrl.Rule(vacancy_rate['low'] & user_type['vip'], recommended_area['area_b'])
rule10 = ctrl.Rule(vacancy_rate['low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_a'])
rule11 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['regular'] | user_type['member']), 
                 recommended_area['area_e'])
rule12 = ctrl.Rule(vacancy_rate['very_low'] & user_type['vip'], recommended_area['area_d'])
rule13 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_c'])
rule14 = ctrl.Rule(weather_condition['snow'] & user_type['disabled'], recommended_area['area_a'])
rule15 = ctrl.Rule(weather_condition['heavy_rain'] & user_type['disabled'], recommended_area['area_a'])

# Rules for Estimated Waiting Time
rule16 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_high'], waiting_time['very_short'])
rule17 = ctrl.Rule(traffic_density['low'] & vacancy_rate['high'], waiting_time['very_short'])
rule18 = ctrl.Rule(traffic_density['low'] & vacancy_rate['medium'], waiting_time['short'])
rule19 = ctrl.Rule(traffic_density['low'] & vacancy_rate['low'], waiting_time['medium'])
rule20 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_low'], waiting_time['long'])

rule21 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_high'], waiting_time['short'])
rule22 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['high'], waiting_time['short'])
rule23 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['medium'], waiting_time['medium'])
rule24 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['low'], waiting_time['long'])
rule25 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_low'], waiting_time['very_long'])

rule26 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_high'], waiting_time['medium'])
rule27 = ctrl.Rule(traffic_density['high'] & vacancy_rate['high'], waiting_time['medium'])
rule28 = ctrl.Rule(traffic_density['high'] & vacancy_rate['medium'], waiting_time['long'])
rule29 = ctrl.Rule(traffic_density['high'] & vacancy_rate['low'], waiting_time['very_long'])
rule30 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_low'], waiting_time['very_long'])

# Time-based rules
rule31 = ctrl.Rule((time_of_day['morning'] | time_of_day['afternoon']) & traffic_density['high'], 
                 waiting_time['very_long'])
rule32 = ctrl.Rule(weather_condition['heavy_rain'] | weather_condition['snow'], waiting_time['long'])
rule33 = ctrl.Rule(time_of_day['night'] & ~vacancy_rate['very_low'], waiting_time['very_short'])

rule34 = ctrl.Rule((time_of_day['morning']) & (vacancy_rate['high'] | vacancy_rate['very_high']), 
                 recommended_area['area_a'])
rule35 = ctrl.Rule(time_of_day['evening'] & traffic_density['high'], waiting_time['very_long'])
rule36 = ctrl.Rule(time_of_day['night'], recommended_area['area_a'])

# Create control system
parking_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20,
    rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28, rule29, rule30,
    rule31, rule32, rule33, rule34, rule35, rule36
])

# Create simulator
parking_simulator = ctrl.ControlSystemSimulation(parking_ctrl)

# Function to use the parking guidance system
def get_parking_recommendation(traffic_density_val, time_of_day_val, weather_condition_val, 
                             vacancy_rate_val, user_type_val):
    """
    Get parking recommendation based on input variables
    
    Args:
        traffic_density_val: Traffic density value (0-100%)
        time_of_day_val: Time of day (0-24 hours)
        weather_condition_val: Weather condition (0-10)
        vacancy_rate_val: Vacancy rate (0-100%)
        user_type_val: User type (1-5)
    
    Returns:
        dict: Dictionary containing recommended parking area and estimated waiting time
    """
    # Input values to the fuzzy control system
    parking_simulator.input['traffic_density'] = traffic_density_val
    parking_simulator.input['time_of_day'] = time_of_day_val
    parking_simulator.input['weather_condition'] = weather_condition_val
    parking_simulator.input['vacancy_rate'] = vacancy_rate_val
    parking_simulator.input['user_type'] = user_type_val
    
    # Compute result
    try:
        parking_simulator.compute()
        
        # Get output values
        recommended_area_val = parking_simulator.output['recommended_area']
        waiting_time_val = parking_simulator.output['waiting_time']
        
        # Convert area to human-readable format
        if recommended_area_val <= 1.5:
            area_text = "Area A (closest to entrance)"
        elif recommended_area_val <= 2.5:
            area_text = "Area B"
        elif recommended_area_val <= 3.5:
            area_text = "Area C"
        elif recommended_area_val <= 4.5:
            area_text = "Area D"
        else:
            area_text = "Area E (farthest from entrance)"
        
        return {
            'recommended_area_value': recommended_area_val,
            'recommended_area_text': area_text,
            'waiting_time_value': waiting_time_val,
            'waiting_time_text': f"{int(waiting_time_val)} minutes"
        }
    except Exception as e:
        return {
            'error': f'Unable to compute recommendation. Error: {str(e)}'
        }

# Test the system with sample inputs
test_result = get_parking_recommendation(
    traffic_density_val=80,    # High traffic
    time_of_day_val=9,         # Morning rush hour
    weather_condition_val=1,   # Clear weather
    vacancy_rate_val=50,       # Medium vacancy
    user_type_val=1            # Regular user
)
print("Test Result:")
if 'error' in test_result:
    print(f"Error: {test_result['error']}")
else:
    print(f"Recommended Parking: {test_result['recommended_area_text']} (value: {test_result['recommended_area_value']:.2f})")
    print(f"Estimated Waiting Time: {test_result['waiting_time_text']} (value: {test_result['waiting_time_value']:.2f})")

# Create a class to encapsulate the fuzzy logic system for easier integration with GUI
class ParkingGuidanceSystem:
    def __init__(self):
        """Initialize the Parking Guidance System with fuzzy logic control system"""
        self.parking_simulator = parking_simulator
        
    def get_recommendation(self, traffic_density_val, time_of_day_val, weather_condition_val, 
                          vacancy_rate_val, user_type_val):
        """
        Get parking recommendation based on the provided input values
        
        Args:
            traffic_density_val (float): Traffic density value (0-100%)
            time_of_day_val (float): Time of day (0-24 hours)
            weather_condition_val (float): Weather condition (0-10)
            vacancy_rate_val (float): Vacancy rate (0-100%)
            user_type_val (int): User type (1-5)
            
        Returns:
            dict: Dictionary containing recommended parking area and estimated waiting time
        """
        return get_parking_recommendation(
            traffic_density_val, time_of_day_val, weather_condition_val, 
            vacancy_rate_val, user_type_val
        )
    
    def get_input_ranges(self):
        """
        Get the valid ranges for all input variables
        
        Returns:
            dict: Dictionary containing valid ranges for all input variables
        """
        return {
            'traffic_density': {'min': 0, 'max': 100, 'unit': '%'},
            'time_of_day': {'min': 0, 'max': 24, 'unit': 'hours'},
            'weather_condition': {'min': 0, 'max': 10, 'unit': 'scale'},
            'vacancy_rate': {'min': 0, 'max': 100, 'unit': '%'},
            'user_type': {'min': 1, 'max': 5, 'unit': 'type'}
        }
    
    def get_user_type_mapping(self):
        """
        Get mapping for user type values to descriptions
        
        Returns:
            dict: Dictionary mapping user type values to descriptions
        """
        return {
            1: "Regular",
            2: "Member",
            3: "VIP",
            4: "Disabled",
            5: "Staff"
        }
    
    def get_weather_condition_mapping(self):
        """
        Get mapping for weather condition values to descriptions
        
        Returns:
            dict: Dictionary mapping weather condition values to descriptions
        """
        return {
            0: "Clear (0)",
            1: "Clear (1)",
            2: "Clear (2)",
            3: "Light Rain (3)",
            4: "Light Rain (4)",
            5: "Light Rain (5)",
            6: "Heavy Rain (6)",
            7: "Heavy Rain (7)",
            8: "Heavy Rain (8)",
            9: "Snow (9)",
            10: "Snow (10)"
        }

# Create an instance of the parking guidance system for later use
parking_system = ParkingGuidanceSystem()

ModuleNotFoundError: No module named 'skfuzzy'

In [5]:
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Install required packages
await terminal.run('pip install numpy scikit-fuzzy')

# Import necessary libraries
import numpy as np
import skfuzzy as fuzz
from skfuzzy import control as ctrl

# Create input variables for the fuzzy control system based on the analysis document
# Input 1: Traffic Density (0-100%)
traffic_density = ctrl.Antecedent(np.arange(0, 101, 1), 'traffic_density')
# Input 2: Time of Day (0-24 hours)
time_of_day = ctrl.Antecedent(np.arange(0, 25, 1), 'time_of_day')
# Input 3: Weather Condition (0-10)
weather_condition = ctrl.Antecedent(np.arange(0, 11, 1), 'weather_condition')
# Input 4: Vacancy Rate (0-100%)
vacancy_rate = ctrl.Antecedent(np.arange(0, 101, 1), 'vacancy_rate')
# Input 5: User Type (1-5)
user_type = ctrl.Antecedent(np.arange(1, 6, 1), 'user_type')

# Create output variables for the fuzzy control system
# Output 1: Recommended Parking Area (1-5)
recommended_area = ctrl.Consequent(np.arange(1, 6, 1), 'recommended_area')
# Output 2: Estimated Waiting Time (0-30 minutes)
waiting_time = ctrl.Consequent(np.arange(0, 31, 1), 'waiting_time')

# Define membership functions for each input variable
# Traffic Density
traffic_density['low'] = fuzz.trapmf(traffic_density.universe, [0, 0, 20, 40])
traffic_density['medium'] = fuzz.trimf(traffic_density.universe, [30, 50, 70])
traffic_density['high'] = fuzz.trapmf(traffic_density.universe, [60, 80, 100, 100])

# Time of Day
time_of_day['early_morning'] = fuzz.trapmf(time_of_day.universe, [0, 0, 6, 8])
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [7, 9, 11])
time_of_day['noon'] = fuzz.trimf(time_of_day.universe, [10, 12, 14])
time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [13, 15, 18])
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [17, 19, 22])
time_of_day['night'] = fuzz.trapmf(time_of_day.universe, [21, 23, 24, 24])

# Weather Condition
weather_condition['clear'] = fuzz.trapmf(weather_condition.universe, [0, 0, 2, 3])
weather_condition['light_rain'] = fuzz.trimf(weather_condition.universe, [2, 4, 6])
weather_condition['heavy_rain'] = fuzz.trimf(weather_condition.universe, [5, 7, 9])
weather_condition['snow'] = fuzz.trapmf(weather_condition.universe, [8, 9, 10, 10])

# Vacancy Rate
vacancy_rate['very_low'] = fuzz.trapmf(vacancy_rate.universe, [0, 0, 10, 20])
vacancy_rate['low'] = fuzz.trimf(vacancy_rate.universe, [15, 25, 35])
vacancy_rate['medium'] = fuzz.trimf(vacancy_rate.universe, [30, 50, 70])
vacancy_rate['high'] = fuzz.trimf(vacancy_rate.universe, [60, 75, 90])
vacancy_rate['very_high'] = fuzz.trapmf(vacancy_rate.universe, [85, 95, 100, 100])

# User Type
user_type['regular'] = fuzz.trimf(user_type.universe, [1, 1, 2])
user_type['member'] = fuzz.trimf(user_type.universe, [1, 2, 3])
user_type['vip'] = fuzz.trimf(user_type.universe, [2, 3, 4])
user_type['disabled'] = fuzz.trimf(user_type.universe, [3, 4, 5])
user_type['staff'] = fuzz.trimf(user_type.universe, [4, 5, 5])

# Define membership functions for each output variable
# Recommended Parking Area
recommended_area['area_a'] = fuzz.trimf(recommended_area.universe, [1, 1, 2])  # Closest to entrance
recommended_area['area_b'] = fuzz.trimf(recommended_area.universe, [1, 2, 3])
recommended_area['area_c'] = fuzz.trimf(recommended_area.universe, [2, 3, 4])
recommended_area['area_d'] = fuzz.trimf(recommended_area.universe, [3, 4, 5])
recommended_area['area_e'] = fuzz.trimf(recommended_area.universe, [4, 5, 5])  # Farthest from entrance

# Estimated Waiting Time
waiting_time['very_short'] = fuzz.trapmf(waiting_time.universe, [0, 0, 2, 5])
waiting_time['short'] = fuzz.trimf(waiting_time.universe, [3, 7, 11])
waiting_time['medium'] = fuzz.trimf(waiting_time.universe, [9, 13, 17])
waiting_time['long'] = fuzz.trimf(waiting_time.universe, [15, 20, 25])
waiting_time['very_long'] = fuzz.trapmf(waiting_time.universe, [23, 28, 30, 30])

# Define fuzzy rules for the control system

# Rules for Recommended Parking Area
rule1 = ctrl.Rule(vacancy_rate['very_high'], recommended_area['area_a'])
rule2 = ctrl.Rule(vacancy_rate['high'] & user_type['regular'], recommended_area['area_b'])
rule3 = ctrl.Rule(vacancy_rate['high'] & ~user_type['regular'], recommended_area['area_a'])
rule4 = ctrl.Rule(vacancy_rate['medium'] & user_type['regular'], recommended_area['area_c'])
rule5 = ctrl.Rule(vacancy_rate['medium'] & user_type['member'], recommended_area['area_b'])
rule6 = ctrl.Rule(vacancy_rate['medium'] & (user_type['vip'] | user_type['disabled'] | user_type['staff']), 
                recommended_area['area_a'])
rule7 = ctrl.Rule(vacancy_rate['low'] & user_type['regular'], recommended_area['area_d'])
rule8 = ctrl.Rule(vacancy_rate['low'] & user_type['member'], recommended_area['area_c'])
rule9 = ctrl.Rule(vacancy_rate['low'] & user_type['vip'], recommended_area['area_b'])
rule10 = ctrl.Rule(vacancy_rate['low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_a'])
rule11 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['regular'] | user_type['member']), 
                 recommended_area['area_e'])
rule12 = ctrl.Rule(vacancy_rate['very_low'] & user_type['vip'], recommended_area['area_d'])
rule13 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_c'])
rule14 = ctrl.Rule(weather_condition['snow'] & user_type['disabled'], recommended_area['area_a'])
rule15 = ctrl.Rule(weather_condition['heavy_rain'] & user_type['disabled'], recommended_area['area_a'])

# Rules for Estimated Waiting Time
rule16 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_high'], waiting_time['very_short'])
rule17 = ctrl.Rule(traffic_density['low'] & vacancy_rate['high'], waiting_time['very_short'])
rule18 = ctrl.Rule(traffic_density['low'] & vacancy_rate['medium'], waiting_time['short'])
rule19 = ctrl.Rule(traffic_density['low'] & vacancy_rate['low'], waiting_time['medium'])
rule20 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_low'], waiting_time['long'])

rule21 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_high'], waiting_time['short'])
rule22 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['high'], waiting_time['short'])
rule23 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['medium'], waiting_time['medium'])
rule24 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['low'], waiting_time['long'])
rule25 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_low'], waiting_time['very_long'])

rule26 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_high'], waiting_time['medium'])
rule27 = ctrl.Rule(traffic_density['high'] & vacancy_rate['high'], waiting_time['medium'])
rule28 = ctrl.Rule(traffic_density['high'] & vacancy_rate['medium'], waiting_time['long'])
rule29 = ctrl.Rule(traffic_density['high'] & vacancy_rate['low'], waiting_time['very_long'])
rule30 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_low'], waiting_time['very_long'])

# Time-based rules
rule31 = ctrl.Rule((time_of_day['morning'] | time_of_day['afternoon']) & traffic_density['high'], 
                 waiting_time['very_long'])
rule32 = ctrl.Rule(weather_condition['heavy_rain'] | weather_condition['snow'], waiting_time['long'])
rule33 = ctrl.Rule(time_of_day['night'] & ~vacancy_rate['very_low'], waiting_time['very_short'])

rule34 = ctrl.Rule((time_of_day['morning']) & (vacancy_rate['high'] | vacancy_rate['very_high']), 
                 recommended_area['area_a'])
rule35 = ctrl.Rule(time_of_day['evening'] & traffic_density['high'], waiting_time['very_long'])
rule36 = ctrl.Rule(time_of_day['night'], recommended_area['area_a'])

# Create control system
parking_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20,
    rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28, rule29, rule30,
    rule31, rule32, rule33, rule34, rule35, rule36
])

# Create simulator
parking_simulator = ctrl.ControlSystemSimulation(parking_ctrl)

# Function to use the parking guidance system
def get_parking_recommendation(traffic_density_val, time_of_day_val, weather_condition_val, 
                             vacancy_rate_val, user_type_val):
    """
    Get parking recommendation based on input variables
    
    Args:
        traffic_density_val: Traffic density value (0-100%)
        time_of_day_val: Time of day (0-24 hours)
        weather_condition_val: Weather condition (0-10)
        vacancy_rate_val: Vacancy rate (0-100%)
        user_type_val: User type (1-5)
    
    Returns:
        dict: Dictionary containing recommended parking area and estimated waiting time
    """
    # Input values to the fuzzy control system
    parking_simulator.input['traffic_density'] = traffic_density_val
    parking_simulator.input['time_of_day'] = time_of_day_val
    parking_simulator.input['weather_condition'] = weather_condition_val
    parking_simulator.input['vacancy_rate'] = vacancy_rate_val
    parking_simulator.input['user_type'] = user_type_val
    
    # Compute result
    try:
        parking_simulator.compute()
        
        # Get output values
        recommended_area_val = parking_simulator.output['recommended_area']
        waiting_time_val = parking_simulator.output['waiting_time']
        
        # Convert area to human-readable format
        if recommended_area_val <= 1.5:
            area_text = "Area A (closest to entrance)"
        elif recommended_area_val <= 2.5:
            area_text = "Area B"
        elif recommended_area_val <= 3.5:
            area_text = "Area C"
        elif recommended_area_val <= 4.5:
            area_text = "Area D"
        else:
            area_text = "Area E (farthest from entrance)"
        
        return {
            'recommended_area_value': recommended_area_val,
            'recommended_area_text': area_text,
            'waiting_time_value': waiting_time_val,
            'waiting_time_text': f"{int(waiting_time_val)} minutes"
        }
    except Exception as e:
        return {
            'error': f'Unable to compute recommendation. Error: {str(e)}'
        }

# Create a class to encapsulate the fuzzy logic system for easier integration with GUI
class ParkingGuidanceSystem:
    def __init__(self):
        """Initialize the Parking Guidance System with fuzzy logic control system"""
        self.parking_simulator = parking_simulator
        
    def get_recommendation(self, traffic_density_val, time_of_day_val, weather_condition_val, 
                          vacancy_rate_val, user_type_val):
        """
        Get parking recommendation based on the provided input values
        
        Args:
            traffic_density_val (float): Traffic density value (0-100%)
            time_of_day_val (float): Time of day (0-24 hours)
            weather_condition_val (float): Weather condition (0-10)
            vacancy_rate_val (float): Vacancy rate (0-100%)
            user_type_val (int): User type (1-5)
            
        Returns:
            dict: Dictionary containing recommended parking area and estimated waiting time
        """
        return get_parking_recommendation(
            traffic_density_val, time_of_day_val, weather_condition_val, 
            vacancy_rate_val, user_type_val
        )
    
    def get_input_ranges(self):
        """
        Get the valid ranges for all input variables
        
        Returns:
            dict: Dictionary containing valid ranges for all input variables
        """
        return {
            'traffic_density': {'min': 0, 'max': 100, 'unit': '%'},
            'time_of_day': {'min': 0, 'max': 24, 'unit': 'hours'},
            'weather_condition': {'min': 0, 'max': 10, 'unit': 'scale'},
            'vacancy_rate': {'min': 0, 'max': 100, 'unit': '%'},
            'user_type': {'min': 1, 'max': 5, 'unit': 'type'}
        }
    
    def get_user_type_mapping(self):
        """
        Get mapping for user type values to descriptions
        
        Returns:
            dict: Dictionary mapping user type values to descriptions
        """
        return {
            1: "Regular",
            2: "Member",
            3: "VIP",
            4: "Disabled",
            5: "Staff"
        }
    
    def get_weather_condition_mapping(self):
        """
        Get mapping for weather condition values to descriptions
        
        Returns:
            dict: Dictionary mapping weather condition values to descriptions
        """
        return {
            0: "Clear (0)",
            1: "Clear (1)",
            2: "Clear (2)",
            3: "Light Rain (3)",
            4: "Light Rain (4)",
            5: "Light Rain (5)",
            6: "Heavy Rain (6)",
            7: "Heavy Rain (7)",
            8: "Heavy Rain (8)",
            9: "Snow (9)",
            10: "Snow (10)"
        }

# Test the system with sample inputs
try:
    test_result = get_parking_recommendation(
        traffic_density_val=80,    # High traffic
        time_of_day_val=9,         # Morning rush hour
        weather_condition_val=1,   # Clear weather
        vacancy_rate_val=50,       # Medium vacancy
        user_type_val=1            # Regular user
    )
    print("Test Result:")
    if 'error' in test_result:
        print(f"Error: {test_result['error']}")
    else:
        print(f"Recommended Parking: {test_result['recommended_area_text']} (value: {test_result['recommended_area_value']:.2f})")
        print(f"Estimated Waiting Time: {test_result['waiting_time_text']} (value: {test_result['waiting_time_value']:.2f})")
except Exception as e:
    print(f"Error during test: {str(e)}")

# Create an instance of the parking guidance system for later use
parking_system = ParkingGuidanceSystem()

ModuleNotFoundError: No module named 'skfuzzy'

In [6]:
# First, let's properly install the necessary packages
from metagpt.tools.libs.terminal import Terminal
terminal = Terminal()

# Install required packages with explicit version and verification
print("Installing required packages...")
await terminal.run('pip install --upgrade numpy')
await terminal.run('pip install --upgrade scikit-fuzzy')
await terminal.run('pip list | grep fuzzy')

try:
    # Import necessary libraries
    import numpy as np
    import skfuzzy as fuzz
    from skfuzzy import control as ctrl
    print("Successfully imported skfuzzy!")
except ImportError as e:
    print(f"Import error: {e}")
    # Try alternative approach - sometimes the package name is different
    print("Trying alternative package installation...")
    await terminal.run('pip install --upgrade scikit-fuzzy')
    await terminal.run('pip install --upgrade scikit_fuzzy')
    
    # Try importing again
    import numpy as np
    import skfuzzy as fuzz
    from skfuzzy import control as ctrl

# Create input variables for the fuzzy control system based on the analysis document
# Input 1: Traffic Density (0-100%)
traffic_density = ctrl.Antecedent(np.arange(0, 101, 1), 'traffic_density')
# Input 2: Time of Day (0-24 hours)
time_of_day = ctrl.Antecedent(np.arange(0, 25, 1), 'time_of_day')
# Input 3: Weather Condition (0-10)
weather_condition = ctrl.Antecedent(np.arange(0, 11, 1), 'weather_condition')
# Input 4: Vacancy Rate (0-100%)
vacancy_rate = ctrl.Antecedent(np.arange(0, 101, 1), 'vacancy_rate')
# Input 5: User Type (1-5)
user_type = ctrl.Antecedent(np.arange(1, 6, 1), 'user_type')

# Create output variables for the fuzzy control system
# Output 1: Recommended Parking Area (1-5)
recommended_area = ctrl.Consequent(np.arange(1, 6, 1), 'recommended_area')
# Output 2: Estimated Waiting Time (0-30 minutes)
waiting_time = ctrl.Consequent(np.arange(0, 31, 1), 'waiting_time')

# Define membership functions for each input variable
# Traffic Density
traffic_density['low'] = fuzz.trapmf(traffic_density.universe, [0, 0, 20, 40])
traffic_density['medium'] = fuzz.trimf(traffic_density.universe, [30, 50, 70])
traffic_density['high'] = fuzz.trapmf(traffic_density.universe, [60, 80, 100, 100])

# Time of Day
time_of_day['early_morning'] = fuzz.trapmf(time_of_day.universe, [0, 0, 6, 8])
time_of_day['morning'] = fuzz.trimf(time_of_day.universe, [7, 9, 11])
time_of_day['noon'] = fuzz.trimf(time_of_day.universe, [10, 12, 14])
time_of_day['afternoon'] = fuzz.trimf(time_of_day.universe, [13, 15, 18])
time_of_day['evening'] = fuzz.trimf(time_of_day.universe, [17, 19, 22])
time_of_day['night'] = fuzz.trapmf(time_of_day.universe, [21, 23, 24, 24])

# Weather Condition
weather_condition['clear'] = fuzz.trapmf(weather_condition.universe, [0, 0, 2, 3])
weather_condition['light_rain'] = fuzz.trimf(weather_condition.universe, [2, 4, 6])
weather_condition['heavy_rain'] = fuzz.trimf(weather_condition.universe, [5, 7, 9])
weather_condition['snow'] = fuzz.trapmf(weather_condition.universe, [8, 9, 10, 10])

# Vacancy Rate
vacancy_rate['very_low'] = fuzz.trapmf(vacancy_rate.universe, [0, 0, 10, 20])
vacancy_rate['low'] = fuzz.trimf(vacancy_rate.universe, [15, 25, 35])
vacancy_rate['medium'] = fuzz.trimf(vacancy_rate.universe, [30, 50, 70])
vacancy_rate['high'] = fuzz.trimf(vacancy_rate.universe, [60, 75, 90])
vacancy_rate['very_high'] = fuzz.trapmf(vacancy_rate.universe, [85, 95, 100, 100])

# User Type
user_type['regular'] = fuzz.trimf(user_type.universe, [1, 1, 2])
user_type['member'] = fuzz.trimf(user_type.universe, [1, 2, 3])
user_type['vip'] = fuzz.trimf(user_type.universe, [2, 3, 4])
user_type['disabled'] = fuzz.trimf(user_type.universe, [3, 4, 5])
user_type['staff'] = fuzz.trimf(user_type.universe, [4, 5, 5])

# Define membership functions for each output variable
# Recommended Parking Area
recommended_area['area_a'] = fuzz.trimf(recommended_area.universe, [1, 1, 2])  # Closest to entrance
recommended_area['area_b'] = fuzz.trimf(recommended_area.universe, [1, 2, 3])
recommended_area['area_c'] = fuzz.trimf(recommended_area.universe, [2, 3, 4])
recommended_area['area_d'] = fuzz.trimf(recommended_area.universe, [3, 4, 5])
recommended_area['area_e'] = fuzz.trimf(recommended_area.universe, [4, 5, 5])  # Farthest from entrance

# Estimated Waiting Time
waiting_time['very_short'] = fuzz.trapmf(waiting_time.universe, [0, 0, 2, 5])
waiting_time['short'] = fuzz.trimf(waiting_time.universe, [3, 7, 11])
waiting_time['medium'] = fuzz.trimf(waiting_time.universe, [9, 13, 17])
waiting_time['long'] = fuzz.trimf(waiting_time.universe, [15, 20, 25])
waiting_time['very_long'] = fuzz.trapmf(waiting_time.universe, [23, 28, 30, 30])

# Define fuzzy rules for the control system

# Rules for Recommended Parking Area
rule1 = ctrl.Rule(vacancy_rate['very_high'], recommended_area['area_a'])
rule2 = ctrl.Rule(vacancy_rate['high'] & user_type['regular'], recommended_area['area_b'])
rule3 = ctrl.Rule(vacancy_rate['high'] & ~user_type['regular'], recommended_area['area_a'])
rule4 = ctrl.Rule(vacancy_rate['medium'] & user_type['regular'], recommended_area['area_c'])
rule5 = ctrl.Rule(vacancy_rate['medium'] & user_type['member'], recommended_area['area_b'])
rule6 = ctrl.Rule(vacancy_rate['medium'] & (user_type['vip'] | user_type['disabled'] | user_type['staff']), 
                recommended_area['area_a'])
rule7 = ctrl.Rule(vacancy_rate['low'] & user_type['regular'], recommended_area['area_d'])
rule8 = ctrl.Rule(vacancy_rate['low'] & user_type['member'], recommended_area['area_c'])
rule9 = ctrl.Rule(vacancy_rate['low'] & user_type['vip'], recommended_area['area_b'])
rule10 = ctrl.Rule(vacancy_rate['low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_a'])
rule11 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['regular'] | user_type['member']), 
                 recommended_area['area_e'])
rule12 = ctrl.Rule(vacancy_rate['very_low'] & user_type['vip'], recommended_area['area_d'])
rule13 = ctrl.Rule(vacancy_rate['very_low'] & (user_type['disabled'] | user_type['staff']), 
                 recommended_area['area_c'])
rule14 = ctrl.Rule(weather_condition['snow'] & user_type['disabled'], recommended_area['area_a'])
rule15 = ctrl.Rule(weather_condition['heavy_rain'] & user_type['disabled'], recommended_area['area_a'])

# Rules for Estimated Waiting Time
rule16 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_high'], waiting_time['very_short'])
rule17 = ctrl.Rule(traffic_density['low'] & vacancy_rate['high'], waiting_time['very_short'])
rule18 = ctrl.Rule(traffic_density['low'] & vacancy_rate['medium'], waiting_time['short'])
rule19 = ctrl.Rule(traffic_density['low'] & vacancy_rate['low'], waiting_time['medium'])
rule20 = ctrl.Rule(traffic_density['low'] & vacancy_rate['very_low'], waiting_time['long'])

rule21 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_high'], waiting_time['short'])
rule22 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['high'], waiting_time['short'])
rule23 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['medium'], waiting_time['medium'])
rule24 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['low'], waiting_time['long'])
rule25 = ctrl.Rule(traffic_density['medium'] & vacancy_rate['very_low'], waiting_time['very_long'])

rule26 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_high'], waiting_time['medium'])
rule27 = ctrl.Rule(traffic_density['high'] & vacancy_rate['high'], waiting_time['medium'])
rule28 = ctrl.Rule(traffic_density['high'] & vacancy_rate['medium'], waiting_time['long'])
rule29 = ctrl.Rule(traffic_density['high'] & vacancy_rate['low'], waiting_time['very_long'])
rule30 = ctrl.Rule(traffic_density['high'] & vacancy_rate['very_low'], waiting_time['very_long'])

# Time-based rules
rule31 = ctrl.Rule((time_of_day['morning'] | time_of_day['afternoon']) & traffic_density['high'], 
                 waiting_time['very_long'])
rule32 = ctrl.Rule(weather_condition['heavy_rain'] | weather_condition['snow'], waiting_time['long'])
rule33 = ctrl.Rule(time_of_day['night'] & ~vacancy_rate['very_low'], waiting_time['very_short'])

rule34 = ctrl.Rule((time_of_day['morning']) & (vacancy_rate['high'] | vacancy_rate['very_high']), 
                 recommended_area['area_a'])
rule35 = ctrl.Rule(time_of_day['evening'] & traffic_density['high'], waiting_time['very_long'])
rule36 = ctrl.Rule(time_of_day['night'], recommended_area['area_a'])

# Create control system
parking_ctrl = ctrl.ControlSystem([
    rule1, rule2, rule3, rule4, rule5, rule6, rule7, rule8, rule9, rule10,
    rule11, rule12, rule13, rule14, rule15, rule16, rule17, rule18, rule19, rule20,
    rule21, rule22, rule23, rule24, rule25, rule26, rule27, rule28, rule29, rule30,
    rule31, rule32, rule33, rule34, rule35, rule36
])

# Create simulator
parking_simulator = ctrl.ControlSystemSimulation(parking_ctrl)

# Function to use the parking guidance system
def get_parking_recommendation(traffic_density_val, time_of_day_val, weather_condition_val, 
                             vacancy_rate_val, user_type_val):
    """
    Get parking recommendation based on input variables
    
    Args:
        traffic_density_val: Traffic density value (0-100%)
        time_of_day_val: Time of day (0-24 hours)
        weather_condition_val: Weather condition (0-10)
        vacancy_rate_val: Vacancy rate (0-100%)
        user_type_val: User type (1-5)
    
    Returns:
        dict: Dictionary containing recommended parking area and estimated waiting time
    """
    # Input values to the fuzzy control system
    parking_simulator.input['traffic_density'] = traffic_density_val
    parking_simulator.input['time_of_day'] = time_of_day_val
    parking_simulator.input['weather_condition'] = weather_condition_val
    parking_simulator.input['vacancy_rate'] = vacancy_rate_val
    parking_simulator.input['user_type'] = user_type_val
    
    # Compute result
    try:
        parking_simulator.compute()
        
        # Get output values
        recommended_area_val = parking_simulator.output['recommended_area']
        waiting_time_val = parking_simulator.output['waiting_time']
        
        # Convert area to human-readable format
        if recommended_area_val <= 1.5:
            area_text = "Area A (closest to entrance)"
        elif recommended_area_val <= 2.5:
            area_text = "Area B"
        elif recommended_area_val <= 3.5:
            area_text = "Area C"
        elif recommended_area_val <= 4.5:
            area_text = "Area D"
        else:
            area_text = "Area E (farthest from entrance)"
        
        return {
            'recommended_area_value': recommended_area_val,
            'recommended_area_text': area_text,
            'waiting_time_value': waiting_time_val,
            'waiting_time_text': f"{int(waiting_time_val)} minutes"
        }
    except Exception as e:
        return {
            'error': f'Unable to compute recommendation. Error: {str(e)}'
        }

# Create a class to encapsulate the fuzzy logic system for easier integration with GUI
class ParkingGuidanceSystem:
    def __init__(self):
        """Initialize the Parking Guidance System with fuzzy logic control system"""
        self.parking_simulator = parking_simulator
        
    def get_recommendation(self, traffic_density_val, time_of_day_val, weather_condition_val, 
                          vacancy_rate_val, user_type_val):
        """
        Get parking recommendation based on the provided input values
        
        Args:
            traffic_density_val (float): Traffic density value (0-100%)
            time_of_day_val (float): Time of day (0-24 hours)
            weather_condition_val (float): Weather condition (0-10)
            vacancy_rate_val (float): Vacancy rate (0-100%)
            user_type_val (int): User type (1-5)
            
        Returns:
            dict: Dictionary containing recommended parking area and estimated waiting time
        """
        return get_parking_recommendation(
            traffic_density_val, time_of_day_val, weather_condition_val, 
            vacancy_rate_val, user_type_val
        )
    
    def get_input_ranges(self):
        """
        Get the valid ranges for all input variables
        
        Returns:
            dict: Dictionary containing valid ranges for all input variables
        """
        return {
            'traffic_density': {'min': 0, 'max': 100, 'unit': '%'},
            'time_of_day': {'min': 0, 'max': 24, 'unit': 'hours'},
            'weather_condition': {'min': 0, 'max': 10, 'unit': 'scale'},
            'vacancy_rate': {'min': 0, 'max': 100, 'unit': '%'},
            'user_type': {'min': 1, 'max': 5, 'unit': 'type'}
        }
    
    def get_user_type_mapping(self):
        """
        Get mapping for user type values to descriptions
        
        Returns:
            dict: Dictionary mapping user type values to descriptions
        """
        return {
            1: "Regular",
            2: "Member",
            3: "VIP",
            4: "Disabled",
            5: "Staff"
        }
    
    def get_weather_condition_mapping(self):
        """
        Get mapping for weather condition values to descriptions
        
        Returns:
            dict: Dictionary mapping weather condition values to descriptions
        """
        return {
            0: "Clear (0)",
            1: "Clear (1)",
            2: "Clear (2)",
            3: "Light Rain (3)",
            4: "Light Rain (4)",
            5: "Light Rain (5)",
            6: "Heavy Rain (6)",
            7: "Heavy Rain (7)",
            8: "Heavy Rain (8)",
            9: "Snow (9)",
            10: "Snow (10)"
        }

# For testing purposes, let's try a direct approach without using skfuzzy
# if we encounter import errors during execution
try:
    # Test the system with sample inputs
    test_result = get_parking_recommendation(
        traffic_density_val=80,    # High traffic
        time_of_day_val=9,         # Morning rush hour
        weather_condition_val=1,   # Clear weather
        vacancy_rate_val=50,       # Medium vacancy
        user_type_val=1            # Regular user
    )
    print("Test Result:")
    if 'error' in test_result:
        print(f"Error: {test_result['error']}")
    else:
        print(f"Recommended Parking: {test_result['recommended_area_text']} (value: {test_result['recommended_area_value']:.2f})")
        print(f"Estimated Waiting Time: {test_result['waiting_time_text']} (value: {test_result['waiting_time_value']:.2f})")
except Exception as e:
    print(f"Error during test: {str(e)}")

# Create an instance of the parking guidance system for later use
parking_system = ParkingGuidanceSystem()

print("Parking guidance system implementation complete!")

Installing required packages...
Import error: No module named 'skfuzzy'
Trying alternative package installation...


ModuleNotFoundError: No module named 'skfuzzy'